In [1]:
# module that imports CSVs and processes to DB
import os, errno, array, csv, math, re
from datetime import datetime
import dateutil.parser
import importlib
import psycopg2
import fieldmaps

In [3]:
# Alabama
import fieldmaps.al
state = fieldmaps.State('AL', webdate = True)
well_source_url = 'http://www.gsa.state.al.us/ogb/results.aspx?api=%s&permit=%s'
state.source_url = 'http://www.gsa.state.al.us/ogb/db_main.html'
state.description = """
Data from the Geological Survey of Alabama State Oil and Gas Board
"""
state.write_to_db()

dataset = fieldmaps.Dataset(state, fieldmaps.al.layout)
db_result = dataset.write_to_db()

Wrote description for AL to database
16414 of 17086 rows loaded
first item: [{'state': 'AL', 'api': '01297203230000', 'lon': -88.302816, 'lat': 30.245575, 'date': datetime.date(1991, 7, 29), 'type': 'GAS', 'status': 'PA', 'category': 'CONVENTIONAL', 'description': 'Permit Num: 10008-OS-52-BH\nWell Name: STATE LEASE 527 #1\nOperator: Arco Oil & Gas Co., Div. Of Atlantic Richfield'}]
loaded 16414 of 16414 records into the database


In [5]:
# Alaska
import fieldmaps.ak
state = fieldmaps.State('AK')
state.projection = 4267 # NAD27
state.source_url = 'http://aogweb.state.ak.us/DataMiner3/Forms/WellList.aspx'
state.description = """
Data from the Alaska Oil and Gas Conservation Commission
"""
state.write_to_db()

dataset = fieldmaps.Dataset(state, fieldmaps.ak.layout)
dataset.write_to_db()

Wrote description for AK to database
8911 of 9160 rows loaded
first item: [{'state': 'AK', 'api': '50103205776300', 'lon': -151.043238764074, 'lat': 70.3376726847063, 'date': datetime.date(2018, 3, 13), 'type': 'OIL', 'status': 'UNKNOWN', 'category': 'CONVENTIONAL', 'description': 'Permit No. 218026\nOperator: CONOCOPHILLIPS ALASKA, INC.\nArea: ARCTIC SLOPE\nField: COLVILLE RIVER\nPools: QANNIK  OIL\nWell Name: COLVILLE RIV QANN CD2-463L2-01'}]
loaded 8911 of 8911 records into the database


8911

In [7]:
# Arkansas
import fieldmaps.ar
state = fieldmaps.State('AR')
state.source_url = 'http://www.noggateway.org/reports'
state.description = "Data from the National Oil and Gas Gateway"
state.write_to_db()

state.projection = 4269 # NAD83

dataset = fieldmaps.Dataset(state, fieldmaps.ar.layout)
dataset.write_to_db()

Wrote description for AR to database
30318 of 52490 rows loaded
first item: [{'state': 'AR', 'api': '03131110170000', 'lon': -94.2957, 'lat': 35.2014, 'date': datetime.date(2003, 9, 8), 'type': 'GAS', 'status': 'ACTIVE', 'category': 'CONVENTIONAL', 'description': 'WELL NAME: Harkreader  2-15\nWELL NUMBER: 2-15\nOPERATOR: Xto Energy, Inc.\nCOUNTY: Sebastian\nFIELD NAME: B-44 Area'}]
loaded 30318 of 30318 records into the database


30318

In [10]:
# Arizona
import fieldmaps.az
state = fieldmaps.State('AZ')
state.source_url = 'http://services.azgs.az.gov/arcgis/rest/services/aasggeothermal/AZWellHeaders/MapServer/0'
state.description = """
Data from the Arizona Oil and Gas Conservation Commission ArcGIS map at http://ogviewer.azdeq.gov/.

Dates do not seem to be available from this source.
"""
state.date_function = lambda x: None
state.write_to_db()

dataset = fieldmaps.Dataset(state, fieldmaps.az.layout)
db_result = dataset.write_to_db()

Wrote description for AZ to database
1144 of 4774 rows loaded
first item: [{'state': 'AZ', 'api': '0201290001', 'lon': -113.6326688, 'lat': 33.49979433, 'date': None, 'type': 'TEST', 'status': 'PA', 'category': None, 'description': 'Well Name: Phillips Petroleum H01 Federal\nOperator: Phillips Petroleum\nCounty: LA PAZ\nField: \nAZ OGCC Notes: no logs'}]
loaded 1144 of 1144 records into the database


In [11]:
# California
import fieldmaps.ca
state = fieldmaps.State('CA', filename = 'california-data.csv', prepend_api = True, webdate = True)
state.source_url = 'http://spatialservices.conservation.ca.gov/arcgis/rest/services/DOMS/Wells/MapServer/0'
state.description = """California Division of Oil, Gas, & Geothermal Resources 
at http://www.conservation.ca.gov/dog. Type codes: ftp://ftp.consrv.ca.gov/pub/oil/GIS/ReadMe.pdf"""
state.write_to_db()

dataset = fieldmaps.Dataset(state, fieldmaps.ca.layout)
dataset.write_to_db()

Wrote description for CA to database
226634 of 226634 rows loaded
first item: [{'state': 'CA', 'api': '0402900001', 'lon': -118.95876, 'lat': 35.43406, 'date': datetime.date(2008, 11, 29), 'type': 'OILANDGAS', 'status': 'ACTIVE', 'category': None, 'description': 'LeaseName: \nOpName: Chevron U.S.A. Inc.\nURL: https://secure.conservation.ca.gov/WellSearch/Details?api=02900001\nComments: \nFieldName: Kern River'}]
loaded 226634 of 226634 records into the database


226634

In [12]:
# Colorado
import fieldmaps.co
state = fieldmaps.State('CO')

wells = fieldmaps.Dataset(state, fieldmaps.co.layout)
wells.write_to_db()

114537 of 114537 rows loaded
first item: [{'state': 'CO', 'api': '0500105000', 'lon': -103.727484, 'lat': 39.741587, 'date': datetime.date(1957, 10, 15), 'type': 'DRY HOLE', 'status': 'A', 'category': None, 'description': 'Operator TOMBERLIN* BILL\nWell_Title 1 UPRR-JOLLY'}]
loaded 114537 of 114537 records into the database


114537

In [ ]:
# Connecticut - No Oil or Gas Drilling

In [ ]:
# Delaware - No Oil or Gas Drilling

In [ ]:
# District of Columbia - No Oil or Gas Drilling

In [13]:
# Florida
state = fieldmaps.State('FL', webdate = True,  prepend_api = True)
import fieldmaps.fl

wells = fieldmaps.Dataset(state, fieldmaps.fl.layout, quiet = True)
wells.write_to_db()

1267 of 1444 rows loaded
first item: [{'state': 'FL', 'api': '09912310003', 'lon': -83.45759326189415, 'lat': 30.132361018336038, 'date': datetime.date(1949, 12, 8), 'type': 'DRY HOLE', 'status': 'A', 'category': None, 'description': None}]
loaded 1267 of 1267 records into the database


1267

In [ ]:
# Georgia - No oil or gas activity

In [ ]:
# Hawaii - No oil or gas activity

In [14]:
# Idaho
state = fieldmaps.State('ID')
# import fieldmaps.id
state.load_data()
layout = {
    'api_field': 'API',
    'date_field': 'Date',
    'lat_field': 'lattitudeWGS84',
    'lon_field': 'longitudeWGS84',
    'status_field': 'Status',
    'status_map': {
        'Not Drilled - Permit Expired': 'CANCELLED',
        'Permitted Well': 'PERMITTED',
        'Expired Permit': 'CANCELLED',
        'Plugged and Abandoned': 'PA',
        'Shut In': 'SI',
        'Not Drilled': 'PERMITTED',
        'Drilling': 'ACTIVE',
        'Producing': 'ACTIVE',
        'Completed': 'ACTIVE',
    },
    'type_field': 'Well Type',
    'type_map': {
        '': 'UNKNOWN',
        'Expired Location': 'UNKNOWN',
        'Oil Well': 'OIL',
        'Oil & Gas': 'OILANDGAS',
        'Gas Well': 'GAS',
        'Dry Hole': 'DRY HOLE',
    }
}
wells = fieldmaps.Dataset(state, layout, quiet = True)
wells.write_to_db()

171 of 200 rows loaded
first item: [{'state': 'ID', 'api': '1108100001', 'lon': -111.22, 'lat': 43.8, 'date': datetime.date(1903, 1, 16), 'type': 'UNKNOWN', 'status': 'PA', 'category': None, 'description': None}]
loaded 171 of 171 records into the database


171

In [15]:
# Illinois
state = fieldmaps.State('IL', webdate = True, filename = 'il-data-old.csv')
import fieldmaps.il
wells = fieldmaps.Dataset(state, fieldmaps.il.layout, quiet = True)
wells.write_to_db()

195735 of 195735 rows loaded
first item: [{'state': 'IL', 'api': '120010000200', 'lon': -90.92110000000001, 'lat': 40.080721, 'date': datetime.date(1945, 12, 31), 'type': 'DRY HOLE', 'status': 'A', 'category': None, 'description': None}]
loaded 195735 of 195735 records into the database


195735

In [16]:
# Indiana
state = fieldmaps.State('IN', prepend_api = True)
import fieldmaps.indiana
wells = fieldmaps.Dataset(state, fieldmaps.indiana.layout)
wells.write_to_db()

49741 of 50714 rows loaded
first item: [{'state': 'IN', 'api': '1316735', 'lon': -87.5878048752559, 'lat': 37.87489933916674, 'date': None, 'type': 'OIL', 'status': 'ACTIVE', 'category': None, 'description': 'County: Vanderburgh    \nField Name: Barker                   \nIGS ID: 16735\nIGS Reported Status: Active\nLink: <a href="https://igs.indiana.edu/pdms/wellEvents.cfm?igsID=16735">IGS link</a>'}]
loaded 49741 of 49741 records into the database


49741

In [ ]:
# Iowa - No oil or Gas

In [23]:
# Kansas
state = fieldmaps.State('KS', webdate = True)
state.projection = 4269 # NAD83

import fieldmaps.ks
layout = fieldmaps.ks.layout
wells = fieldmaps.Dataset(state, layout, quiet = True)
wells.write_to_db()

Invalid layout. Available source file columns: 
'', 'API_NUMBER', 'API_WELL_NUMBER', 'API_WORKOVERS', 'COMPLETION_DATE', 'COMPLETION_DATE_TXT', 'COMPLETION_YEAR', 'COUNTY', 'COUNTY_CODE', 'CURR_OPERATOR', 'ELEVATION_DF', 'ELEVATION_GL', 'ELEVATION_KB', 'FEET_EAST_FROM_REFERENCE', 'FEET_NORTH_FROM_REFERENCE', 'FIELD_KID', 'FIELD_NAME', 'KID', 'LEASE_NAME', 'NAD27_LATITUDE', 'NAD27_LONGITUDE', 'NAD83_LATITUDE', 'NAD83_LONGITUDE', 'OBJECTID', 'OPERATOR_KID', 'OPERATOR_NAME', 'ORIG_FID', 'PERMIT_DATE', 'PERMIT_DATE_TXT', 'PLUG_DATE', 'PLUG_DATE_TXT', 'PRINCIPAL_MERIDIAN', 'PRODUCING_FORMATION', 'RANGE', 'RANGE_DIRECTION', 'REFERENCE_CORNER', 'ROTARY_TOTAL_DEPTH', 'SECTION', 'SPOT', 'SPUD_DATE', 'SPUD_DATE_TXT', 'STATE_CODE', 'STATUS', 'STATUS_TXT', 'SUBDIVISION_1_LARGEST', 'SUBDIVISION_2', 'SUBDIVISION_3', 'SUBDIVISION_4_SMALLEST', 'TOWNSHIP', 'TOWNSHIP_DIRECTION', 'WELL_CLASS', 'WELL_LABEL', 'WELL_NAME', 'WELL_TYPE'

Example rows: [OrderedDict([('', '0'), ('API_NUMBER', '15-007-10179'), (

461547

In [40]:
# Kentucky
state = fieldmaps.State('KY')
state.projection = 4267 # NAD27

import fieldmaps.ky
wells = fieldmaps.Dataset(state, fieldmaps.ky.layout)
wells.write_to_db()

117639 of 145745 rows loaded
first item: [{'state': 'KY', 'api': '16195060750000', 'lon': -82.5155, 'lat': 37.3224, 'date': datetime.date(2009, 6, 27), 'type': 'GAS', 'status': 'ACTIVE', 'category': None, 'description': 'WELL NAME: Equitable Production Company 567990\nWELL NUMBER: 567990\nOPERATOR: Eqt Production Company\nCOUNTY: Pike\nFIELD NAME: '}]
loaded 117639 of 117639 records into the database


117639

In [59]:
# Louisiana

state = fieldmaps.State('LA', webdate = True)
import fieldmaps.la

wells = fieldmaps.Dataset(state, fieldmaps.la.layout, quiet = True)
wells.write_to_db()

Invalid layout. Available source file columns: 
'', 'API_NUM', 'AREA_USDW_VALUE', 'CLASSIFICATION', 'COASTAL_PERMIT_NUM', 'COMMENTS', 'DIRWELL', 'DISTRICT_CODE', 'DOC_ACCESS', 'EFFECTIVE_DATE', 'EXEMPT_29E_FLAG', 'FIELD_ID', 'FIELD_NAME', 'FORMATIONS', 'GIS_UPD_DATE', 'HYPERLINK', 'INJECTION_RATE', 'INJECTION_VOLUME', 'INSPECTION_DATE', 'INSPECTION_YEAR_INDEX', 'LAST_RECOMPLETION_DATE', 'LAST_TEST_DATE', 'LEASE_NUM', 'LEGEND', 'LEGEND_DESC', 'LOCATION', 'LOG_REVIEWED_FLAG', 'LOWER_PERFS', 'LUW_CODE', 'LUW_NAME', 'LUW_TYPE_CODE', 'LW_REC_STATUS', 'MAX_SIP', 'MEASURED_DEPTH', 'MERIDIAN', 'MINERAL_INTEREST_FLAG', 'MUD_DENSITY', 'OBJECTID', 'OPERATION_TYPE_CODE', 'ORGANIZATION_ID', 'ORGOP_LINE_ID', 'ORG_OPER_NAME', 'ORIGINAL_COMPLETION_DATE', 'ORPHANED_FLAG', 'ORPHAN_STATUS_CODE', 'ORPHAN_STATUS_DATE', 'PARISH_CODE', 'PARISH_NAME', 'PERMIT_DATE', 'PRODUCT_TYPE_CODE', 'RESERVOIRS', 'SANDS', 'SCOUT_DETAIL', 'SCOUT_MEASURED_DEPTH', 'SCOUT_REPORT_DATE', 'SCOUT_TRUE_VERTICAL_DEPTH', 'SCOUT_WELL

214470

In [42]:
# Maine - No Oil or Gas

In [63]:
# Maryland - gas only
state = fieldmaps.State('MD', prepend_api = True)
layout = {
    'api_field': 'Id',
    'date_field': 'Date_drill',
    'lat_field': 'Y',
    'lon_field': 'X',
    'status_field': 'Status',
    'status_map': {
        '?': 'UNKNOWN',
        'Abandoned': 'A',
        'Active': 'ACTIVE',
        'Plugged': 'PA',
        'Plugged?': 'PA',
        'plugged': 'PA',
    },
    'type_field': 'Status',
    'type_map': {
        '?': 'GAS',
        'Abandoned': 'GAS',
        'Active': 'GAS',
        'Plugged': 'GAS',
        'Plugged?': 'GAS',
        'plugged': 'GAS',
    },
}
wells = fieldmaps.Dataset(state, layout)
wells.write_to_db()

Invalid layout. Available source file columns: 
'X', 'Y', 'Id', 'Date_drill', 'Date_Plugg', 'Depth', 'Formation', 'Permit_Num', 'Permittee', 'Completion', 'Well_Depth', 'Show_of_Ga', 'Common_Nam', 'Status', 'Dry_hole', 'POINT_X', 'POINT_Y', 'Comp_Rpt'

Example rows: [OrderedDict([('X', '-79.37919687'), ('Y', '39.39788269'), ('Id', '0'), ('Date_drill', '7/1/51'), ('Date_Plugg', 'September of 1952'), ('Depth', '3602'), ('Formation', 'Oriskeny'), ('Permit_Num', '7977'), ('Permittee', 'Mid-Atlantic Oil and Gas Co.'), ('Completion', 'yes'), ('Well_Depth', '3602'), ('Show_of_Ga', '3523'), ('Common_Nam', 'Grimes No. 2'), ('Status', 'Plugged'), ('Dry_hole', 'Unknown'), ('POINT_X', '195082.9925'), ('POINT_Y', '194847.897'), ('Comp_Rpt', '<a href="L:/MINING/GIS_Shapefiles/Historic_Reports/hopkins-series.pdf#page=44">Completion Report</a>')]), OrderedDict([('X', '-79.32807733'), ('Y', '39.43129633'), ('Id', '0'), ('Date_drill', '11/9/52'), ('Date_Plugg', 'na'), ('Depth', '5195'), ('Formation', 'O

8

In [44]:
# Massachusetts - No oil or gas

In [68]:
# Michigan
# the date fields appear to be all empty. may require some supplemental scraping
state = fieldmaps.State('MI')

import fieldmaps.mi
wells = fieldmaps.Dataset(state, fieldmaps.mi.layout)
wells.write_to_db()

Invalid layout. Available source file columns: 
'', 'BH_CNTY', 'CNTY_NAME', 'OBJECTID', 'PRUName', 'PRUNumber', 'PermDate', 'PlugDate', 'Slant', 'TRS', 'api_wellno', 'bh_lat', 'bh_long', 'bh_mgr_x', 'bh_mgr_y', 'bore_type', 'cnty_fips', 'co_name', 'completion', 'concen_h2s', 'deep_fm', 'deep_fm_cd', 'dtd', 'elev_ref', 'fed_land', 'fieldname', 'fieldtype', 'kotvd', 'lease_name', 'op_no', 'permit_no', 'prod_fm', 'prod_fm_cd', 'qqqs', 'qqs', 'qs', 'ref_tops', 'sidetrck', 'state_land', 'surf_owner', 'tvd', 'twp_name', 'well_no', 'well_stat', 'well_type'

Example rows: [OrderedDict([('', '0'), ('BH_CNTY', ''), ('CNTY_NAME', ''), ('OBJECTID', '1'), ('PRUName', ''), ('PRUNumber', ''), ('PermDate', ''), ('PlugDate', ''), ('Slant', 'V'), ('TRS', ''), ('api_wellno', '21001003199000'), ('bh_lat', '44.7351'), ('bh_long', '-83.45521'), ('bh_mgr_x', '701317.24'), ('bh_mgr_y', '467974.61'), ('bore_type', 'Pre-Permit'), ('cnty_fips', ''), ('co_name', 'Kurrasch  H A'), ('completion', '0'), ('concen_h2s

76944

In [ ]:
# Minnesota - no oil or gas

In [69]:
# Mississippi
state = fieldmaps.State('MS')
layout = {
   'date_field': ['SPUD DATE', 'FIRST COMPLETION DATE', 'FIRST PRODUCTION DATE'],
    'api_field': 'API NUMBER',
    'type_field': 'STANDARD WELL TYPE DESCRIPTION',
    'status_field': 'STANDARD WELL STATUS DESCRIPTION',
    'lat_field': 'WELLHEAD LATITUDE, DECIMAL DEGREES',
    'lon_field': 'WELLHEAD LONGITUDE, DECIMAL DEGREES',
    'status_map': {
        'Unassigned': 'UNKNOWN',
        'Abandoned': 'A',
        'Abandoned:Plugged': 'PA',
        'Abandoned:Dry Hole': 'A',
        'Active:Producing': 'ACTIVE',
        'Permit:Active': 'ACTIVE',
        'Inactive:Temporarily Abandoned': 'TA',
        'Permit:Cancelled Expired': 'CANCELLED',
        'Active:Completed': 'ACTIVE',
        'Active': 'ACTIVE',
        'Inactive': 'A',
        'Active:Injecting': 'ACTIVE',
    },
    'type_map': {
        'Other': 'OTHER',
        'Gas': 'GAS',
        'Unassigned': 'OTHER',
        'Oil': 'OIL',
        'Observation Monitor Strat Test': 'TEST',
        'Gas Storage': 'STORAGE',
        'Water Supply': 'WATER',
        'Permit': 'UNKNOWN',
        'Underground Injection Control:Class 2 Disposal': 'INJECTION',
        'Dry Hole': 'DRY HOLE',
        'Underground Injection Control:Class 2 Enhanced Recovery': 'INJECTION',
        'Underground Injection Control:Class 1': 'INJECTION',
    },
}
#import fieldmaps.ks
wells = fieldmaps.Dataset(state, layout)
wells.write_to_db()

Invalid layout. Available source file columns: 
'API NUMBER', 'WELL NAME', 'WELL NUMBER', 'OPERATOR', 'STATE', 'COUNTY', 'COMPLETED FORMATION NAME', 'ENCOUNTERED FORMATION NAME', 'STANDARD WELL TYPE DESCRIPTION', 'STANDARD WELL STATUS DESCRIPTION', 'WELLHEAD LATITUDE, DECIMAL DEGREES', 'WELLHEAD LONGITUDE, DECIMAL DEGREES', 'MEASURED DEPTH, FEET', 'TOTAL VERTICAL DEPTH, FEET', 'DRILLING ORIENTATION', 'BASIN NAME', 'FIELD NAME', 'STATE SUBDIVISION', 'FIRST PRODUCTION DATE', 'SPUD DATE', 'FIRST COMPLETION DATE', 'TOP WELL PERFORATION, FEET', 'BOTTOM WELL PERFORATION, FEET', 'FORMATION TOP, FEET', 'FORMATION BOTTOM, FEET', 'IN FRACFOCUS', 'UNDERGROUND SOURCE OF DRINKING WATER', 'LITHOLOGY', 'STATE WELL TYPE', 'STATE WELL STATUS', 'BOTTOM HOLE LATITUDE, DECIMAL DEGREES', 'BOTTOM HOLE LONGITUDE, DECIMAL DEGREES', 'GEODETIC DATUM', 'WELL LOCATION METHOD', 'ELEVATION', 'ELEVATION REFERENCE POINT', 'SECTION', 'TOWNSHIP', 'TOWNSHIP DIRECTION', 'RANGE', 'RANGE DIRECTION', 'MERIDIAN', 'FOOTAGE NO

22033

In [78]:
importlib.reload(fieldmaps)
importlib.reload(fieldmaps.mi)

layout = {
    'api_field': '',
    'date_field': '',
    'lat_field': 'y',
    'lon_field': 'x',
    'status_field': '',
    'status_map': None,
    'type_field': '',
    'type_map': None
}

In [83]:
# Missouri
state = fieldmaps.State('MO', strip_whitespace = ['Well Status', 'Well Type'], prepend_api = True)
#import fieldmaps.ks
layout = {
    'api_field': 'API Number',
    'date_field': ['Spud Date','Well Status Date'],
    'lat_field': 'Well Latitude Decimal',
    'lon_field': 'Well Longitude Decimal',
    'status_field': 'Well Status',
    'status_map': {
        'Abandoned': 'A',
        'Abandoned, Known Location and Verified': 'A',
        'Abandoned, No evidence of existence/ Unable to find': 'A',
        'Abandoned, Unknown Location': 'A',
        'Active Well': 'ACTIVE',
        'Inactive Ð Initial 90 day window': 'TA',
        'Plugged - Approved': 'PA',
        'Proposed Well (Associated with permit application only)': 'UNKNOWN',
        'Shut in - Complete': 'SI',
        'Shut in - Extended': 'SI',
        'Shut in - Incomplete': 'SI',
        'Temporarily Abandoned(Idle)': 'TA',
        'Under Construction': 'PERMITTED',
        'Unknown Well Status': 'UNKNOWN',
        'Water Well Conversion': 'UNKNOWN',
    },
    'type_field': 'Well Type',
    'type_map': {
        'Dry Hole': 'DRY HOLE',
        'Gas Storage Wells': 'STORAGE',
        'Gas(Coalbed Methane)': 'GAS',
        'Gas(Convertional, Commercial)': 'GAS',
        'Gas(Private Use)': 'GAS',
        'Horizontal Oil Well': 'OIL',
        'Injection(Class II Disposal)': 'INJECTION',
        'Injection(Class II EOR)': 'INJECTION',
        'Observation': 'OBSERVATION',
        'Oil': 'OIL',
        'Stratigraphic Test': 'TEST',
        'Stratigraphic Test, None Oil/Gas Related(has geophysical log)': 'TEST',
        'Unknown Well Type': 'UNKNOWN',
        'Water Well': 'WATER',
        'Water Well, None Oil/Gas Related(has geophysical log)': 'WATER',
    },
}
wells = fieldmaps.Dataset(state, layout)
wells.write_to_db()

10287 of 10471 rows loaded
first item: [{'state': 'MO', 'api': '2400100001', 'lon': -92.37907, 'lat': 40.25678, 'date': datetime.date(1939, 9, 1), 'type': 'DRY HOLE', 'status': 'A', 'category': None, 'description': None}]
loaded 10287 of 10287 records into the database


10287

In [ ]:
# Montana
state = fieldmaps.State('MT')
#import fieldmaps.ks
wells = fieldmaps.Dataset(state, )
wells.write_to_db()

In [ ]:
# Nebraska
state = fieldmaps.State('NE')

layout = {
   'date_field': ['SPUD DATE', 'FIRST COMPLETION DATE', 'FIRST PRODUCTION DATE'],
    'api_field': 'API NUMBER',
    'type_field': 'STANDARD WELL TYPE DESCRIPTION',
    'status_field': 'STANDARD WELL STATUS DESCRIPTION',
    'lat_field': 'WELLHEAD LATITUDE, DECIMAL DEGREES',
    'lon_field': 'WELLHEAD LONGITUDE, DECIMAL DEGREES',
    'status_map': None,
    'type_map': None
}
#import fieldmaps.ks
wells = fieldmaps.Dataset(state, layout)
wells.write_to_db()

In [ ]:
# Nevada
state = fieldmaps.State('NV')

In [ ]:
# New Hampshire - no oil or gas

In [ ]:
# New Jersey - no oil or gas

In [ ]:
# New Mexico
state = fieldmaps.State('NM')

In [ ]:
# New York
state = fieldmaps.State('NY')

In [ ]:
# North Carolina - no oil or gas

In [ ]:
# North Dakota
state = fieldmaps.State('ND')

In [ ]:
# Ohio
state = fieldmaps.State('OH')

In [ ]:
# Oklahoma
state = fieldmaps.State('OK')

layout = {
   'date_field': ['SPUD DATE', 'FIRST COMPLETION DATE', 'FIRST PRODUCTION DATE'],
    'api_field': 'API NUMBER',
    'type_field': 'STANDARD WELL TYPE DESCRIPTION',
    'status_field': 'STANDARD WELL STATUS DESCRIPTION',
    'lat_field': 'WELLHEAD LATITUDE, DECIMAL DEGREES',
    'lon_field': 'WELLHEAD LONGITUDE, DECIMAL DEGREES',
    'status_map': None,
    'type_map': None
}
#import fieldmaps.ks
wells = fieldmaps.Dataset(state, layout)
wells.write_to_db()

In [ ]:
# Oregon
state = fieldmaps.State('OR')

In [ ]:
# Pennsylvania
state = State('PA')
state.source_url = 'http://www.depreportingservices.state.pa.us/ReportServer/Pages/ReportViewer.aspx?/Oil_Gas/Spud_External_Data'
state.description = """
Data downloaded in CSV form based on setting SPUD BEGIN DATE = 1/1/1800 and SPUD END DATE = 12/31/2099
Represents 142,788, of which 110,000 have dates wells.
"""
state.write_to_db()

state.layout = {
    'date_field': 'SPUD_DATE',
    'api_field': 'API',
    'api_includes_state': False,
    'type_field': 'WELL_CODE_DESC',
    'status_field': 'WELL_STATUS',
    'category_field': 'UNCONVENTIONAL',
    'location_src_field': None,
    'lat_field': 'LATITUDE',
    'lon_field': 'LONGITUDE',
    'description_layout': '\n'.join(['County: %s',
                             'Operator Name: %s',
                             'OGO No: %s',
                             'Municipality: %s',
                             'Farm Name: %s',
                             'Well Configuration: %s'
                            ]),
    'description_fields': ['COUNTY', 'OPERATOR', 'OGO_NUM', 'MUNICIPALITY', 'FARM_NAME', 'CONFIGURATION'],
    
    'type_map': {
        'UNDETERMINED': 'OTHER',
        'MULTIPLE WELL BORE TYPE': 'OTHER',
        'OIL': 'OIL',
        'OBSERVATION': 'OBSERVATION',
        'DRY HOLE': 'DRY HOLE',
        'COALBED METHANE': 'GAS',
        'GAS': 'GAS',
        'COMB. OIL&GAS': 'OILANDGAS',
        'STORAGE WELL': 'STORAGE',
        'WASTE DISPOSAL': 'DISPOSAL',
        'INJECTION': 'INJECTION',
        'TEST WELL': 'TEST',
    },

    'status_map': {
        'DEP Abandoned List': 'ORPHAN',
        'Plugged OG Well': 'PA',
        'DEP Orphan List': 'ORPHAN',
        'DEP Plugged': 'PA',
        'Regulatory Inactive Status': 'UNKNOWN',
        'Operator Reported Not Drilled': 'CANCELLED',
        'Proposed But Never Materialized': 'CANCELLED',
        'Active': 'ACTIVE',
        'Abandoned': 'PA',
    },

    'category_map': {
        'Yes': 'UNCONVENTIONAL',
        'No': 'CONVENTIONAL',
    },
    'location_src_map': None
}

state.load_data()
dataset = Dataset(state)
db_result = dataset.commit_to_db()
if db_result:
    print ('loaded', str(db_result), 'of', str(len(dataset.processed_data)), 'records into the database')
else:
    print ("** error loading database! **")

In [ ]:
# Rhode Island - no oil or gas

In [ ]:
# South Carolina - No oil or gas

In [ ]:
# South Dakota
state = fieldmaps.State('SD')

In [ ]:
# Tennessee
state = fieldmaps.State('TN')

state.description = """
Tennessee's Department of Environment & Conservation does not 
doesn't make a lot of details available (no spud date, well status). The dates recorded are the 
Permit Date. Status is set to unknown for all wells. Per their website, 
<i>"This minimal permit information is intended to allow the public to easily track the number and 
locations of oil and gas well permits issued. For more detailed information about a specific well, 
contact the Oil and Gas program staff at 615-687-7109 . Permit files can be accessed by visiting 
the Oil and Gas Section at the Nashville Environmental Field Office in Nashville, TN 37243."</i>
"""
state.source_url = 'http://environment-online.state.tn.us:8080/pls/enf_reports/f?p=9034:34300:0::NO:::'
state.write_to_db()

state.layout = {
    'date_field': 'Permit Date',
    'api_field': 'API No',
    'type_field': 'Purpose af Well',
    'status_field': None,
    'category_field': None,
    'location_src_field': None,
    'lat_field': 'Latitude',
    'lon_field': 'Longitude',
    'description_layout': '\n'.join(['County: %s',
                             'Operator Name: %s',
                             'Permit No: %s',
                             'Well Name and Number: %s',
                             'EFO Name: %s'
                            ]),
    'description_fields': ['County', 'Operator Name', 'Permit No', 'Well Name and Number', 'EFO Name'],
    'type_map': None,
    'status_map': None,
    'category_map': None,
    'type_map': {
        '': 'OTHER',
        'Oil': 'OIL',
        'Oil And Gas': 'OILANDGAS',
        'Mineral Core Test': 'TEST',
        'Gas': 'GAS',
        'NCG - Domestic Use': 'OTHER',
    },
    'location_src_map': None
}

state.load_data()
dataset = Dataset(state)
db_result = dataset.commit_to_db()
if db_result:
    print ('loaded', str(db_result), 'of', str(len(dataset.processed_data)), 'records into the database')
else:
    print ("** error loading database! **")

In [ ]:
# Texas
state = fieldmaps.State('TX')

In [ ]:
# Utah
state = fieldmaps.State('UT')

In [ ]:
# Vermont - no oil or gas

In [ ]:
# Virginia
state = fieldmaps.State('VA')

In [ ]:
# Washington
state = fieldmaps.State('WA')

In [ ]:
# West Virginia
state = fieldmaps.State('WV')

In [ ]:
# Wisconsin - no oil or gas

In [ ]:
# Wyoming
state = State('WY')

In [ ]:
# Alaska Offshore
state = State('AK1')
# Pacific Coast Offshore
state = State('CA1')
# Northern Gulf of Mexico
state = State('TX1')
# Atlantic Coast Offshore
state = State('DC1')
